In [ ]:
import numpy as np
import resource

# Norm function
def norm(v):
    return np.sqrt(np.dot(v, v))

# Conjugate directions method
def conjDir(A, b):
    #initialize a,b,n,x,r0,p,residual
    A = np.array(A, float)
    b = np.array(b, float)
    n = b.size
    x = np.zeros(n)
    r0 = b - A.dot(x)
    p = r0.copy()
    residual = [np.linalg.norm(r0)]
    #do 500 iters
    for i in range(1, n+1):
        #dot products
        Ap = A.dot(p)
        rsquare = r0.dot(r0)
        a = rsquare / p.dot(Ap)
        x = x + a * p
        r1 = r0 - a * Ap
        residual.append(np.linalg.norm(r1))
        beta = r1.dot(r1) / rsquare
        p = r1 + beta * p
        r0 = r1
    return x, i, np.array(residual)

#Guassian elimination method
def gauss(A, b):
    A = np.array(A, float)
    b = np.array(b, float)
    n = b.size
    aug = np.zeros((n, n+1), float)
    aug[:, :n] = A
    for i in range(n):
        aug[i, n] = b[i]
    for i in range(n-1):
        piv = np.argmax(abs(aug[i:, i])) + i
        if piv != i:
            aug[[i, piv]] = aug[[piv, i]]
        for j in range(i+1, n):
            scale = aug[j, i] / aug[i, i]
            aug[j, i:] -= scale * aug[i, i:]
    x = np.zeros(n, float)
    for i in range(n-1, -1, -1):
        x[i] = (aug[i, n] - aug[i, i+1:n].dot(x[i+1:n])) / aug[i, i]
    return x

#Built in Numpy solver for norm proof of concept
#Takes in Matrix A and vector b, outputting solution vector x
#Expected Norm should be of same order as Gaussian elimination method
def solveDir(A, b):
    A = np.array(A, float)
    b = np.array(b, float)
    return np.linalg.solve(A, b)

# Measure memory
def memory(func, *args):
    pre = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    result = func(*args)
    post = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    return result, post - pre

#Main 
if __name__ == "__main__":

    M = np.random.randn(500, 500)
    A = M.T.dot(M) + np.eye(500)
    b = np.random.randn(500)


    (resConj, memConj) = memory(conjDir, A, b)
    xConj, iter, residual = resConj

    (xGauss, mem_ge) = memory(gauss, A, b)
    (x_bs, mem_bs) = memory(solveDir, A, b)


    print("Conjugate Directions: iterations = {iter}, final residual = {residual[-1]:.2e}")
    print("Memory use = {memConj} kb")
    print("Gaussian Elimination: final residual = {norm(A.dot(xGauss)-b):.2e}")
    print("Memory use = {mem_ge} kb")
    print("Built-in solve: final residual = {norm(A.dot(x_bs)-b):.2e}")
    print("Memory use = {mem_bs} kb")



Conjugate Directions: iterations = 500, final residual = 5.71e-10
  Memory use = 16384 kb
Gaussian Elimination: final residual = 1.19e-12
  Memory use = 0 kb
Built-in solve: final residual = 1.39e-12
  Memory use = 0 kb
